# Exercícios

1. Considere observações independentes de uma distribuição $N(\mu, \sigma^2=5^2)$. Escrever um código que estime $\mu$ numericamente para os dados.

a) 20, 28, 39

b) <22, [25,32], >38

c) > 22, [25-32], < 38

2. Considere observações independentes de uma distribuição $P(\lambda)$ - Poisson. Escrever um código que estime $\lambda$ numericamente para os dados.

a) 5, 3, 0, 1, 6, 2, 3, 4, 5, 3

b) 5, <3, 0, 1, >4, [1-3], 3, 4, >3, [1-6]

3. Resolver 1(a) e 2(a) por MQO

www.leg.ufpr.br/~paulojus/dsbd

In [1]:
llint_norm <- Vectorize(function(par, y.p, y.i){
    ## y.p Ã© um vetor de dados pontuais
    ## i.i Ã© uma matriz (n x 2) de dados intervalares
    if(missing(y.p)) ll.p <- 0
    else ll.p <-  sum(dnorm(y.p, mean=par, sd=5, log=TRUE))
    if(missing(y.i)) ll.i <- 0
    else ll.i <- sum(apply(y.i, 1, function(x) log(diff(pnorm(x, mean=par, sd=5)))))
    return(ll.i+ll.p)
}, "par")

In [2]:
llint_pois <- Vectorize(function(par, y.p, y.i){
    ## y.p Ã© um vetor de dados pontuais
    ## i.i Ã© uma matriz (n x 2) de dados intervalares
    if(missing(y.p)) ll.p <- 0
    else ll.p <-  sum(dpois(y.p, lambda=par, log=TRUE))
    if(missing(y.i)) ll.i <- 0
    else ll.i <- sum(apply(y.i, 1, function(x) log(diff(ppois(x, lambda=par)))))
    return(ll.i+ll.p)
}, "par")

# 1.a. 20, 28, 39

In [3]:
yp  <- c(20, 28, 39)
optimize(llint_norm, interval=c(0,50), y.p=yp, maximum=TRUE)

$maximum
[1] 29

$objective
[1] -11.22513

# 1.b. <22,  [25-32] , >38

In [4]:
yi  <- cbind(c(-Inf, 25, 38), c(22, 32, Inf)) 
optimize(llint_norm, interval=c(0,50),y.i=yi, maximum=TRUE)

$maximum
[1] 29.50434

$objective
[1] -6.495191

# 1.c. >22,  [25-32] , <38

In [5]:
yi  <- cbind(c(22, 25, -Inf), c(Inf, 32, 38))
optimize(llint_norm, interval=c(0,50),y.i=yi, maximum=TRUE)

$maximum
[1] 28.98517

$objective
[1] -0.7865337

# 2.a.

In [6]:
yp <- c(5, 3, 0, 1, 6, 2, 3, 4, 5, 3)
optimize(llint_pois, interval=c(0,50), y.p=yp, maximum=TRUE)

$maximum
[1] 3.199992

$objective
[1] -20.17989

# 2.b.

In [7]:
yp <- c(5, 0, 1, 3, 4)
yi <- cbind(c(-Inf, 4, 1, 3, 1), c(3, Inf, 3, Inf, 6))
optimize(llint_pois, interval=c(0,50), y.p=yp, y.i=yi, maximum=TRUE)

$maximum
[1] 3.177063

$objective
[1] -14.66584